In [ ]:
from google.colab import files
files.upload()


Saving extra_chunks.csv to extra_chunks.csv


{'extra_chunks.csv': b'doc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url\r\nLog Management Policy,logging,Log Retention and Forensic Value,"Organizations must define log retention periods based on regulatory, operational, and forensic requirements. Retained logs provide historical evidence, support root-cause analysis, and are essential for legal and compliance investigations.","Kurulu\xc5\x9flar, yasal, operasyonel ve adli gereksinimlere g\xc3\xb6re log saklama s\xc3\xbcrelerini tan\xc4\xb1mlamal\xc4\xb1d\xc4\xb1r. Saklanan loglar, ge\xc3\xa7mi\xc5\x9fe d\xc3\xb6n\xc3\xbck kan\xc4\xb1t sa\xc4\x9flar, k\xc3\xb6k neden analizini destekler ve hukuki ile uyumluluk incelemeleri i\xc3\xa7in kritik \xc3\xb6neme sahiptir.","https://www.sans.org/information-security-policy/log-management-policy/"\r\ndoc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url\r\nLog Management Policy,logging,Log Review and Accountability,"Organizations must define process

In [ ]:
import pandas as pd

df = pd.read_csv("/content/extra_chunks.csv")
print(df.shape)
print(df.columns.tolist())
display(df.head())


(25, 6)
['doc_title', 'policy_type', 'section_title', 'chunk_text_en', 'chunk_text_tr', 'source_url']


,doc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url
0,Log Management Policy,logging,Log Retention and Forensic Value,Organizations must define log retention period...,"Kuruluşlar, yasal, operasyonel ve adli gereksi...",https://www.sans.org/information-security-poli...
1,doc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url
2,Log Management Policy,logging,Log Review and Accountability,Organizations must define processes for regula...,"Kuruluşlar, düzenli log incelemesi için soruml...",https://www.sans.org/information-security-poli...
3,doc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url
4,Log Management Policy,logging,Centralized Logging and Missing Logs,Organizations must maintain systems that aggre...,"Kuruluşlar, tüm bilgi sistemlerinden logları t...",https://www.sans.org/information-security-poli...


In [ ]:
import pandas as pd

df = pd.read_csv("/content/extra_chunks.csv")

# Header'ın veri gibi girdiği satırları sil
df = df[df["doc_title"].astype(str).str.strip().ne("doc_title")].copy()

# index'i düzelt
df = df.reset_index(drop=True)

out_path = "/content/extra_chunks_clean.csv"
df.to_csv(out_path, index=False, encoding="utf-8")

print("✅ Temizlendi:", out_path)
print("Yeni shape:", df.shape)
display(df.head())


✅ Temizlendi: /content/extra_chunks_clean.csv
Yeni shape: (13, 6)


,doc_title,policy_type,section_title,chunk_text_en,chunk_text_tr,source_url
0,Log Management Policy,logging,Log Retention and Forensic Value,Organizations must define log retention period...,"Kuruluşlar, yasal, operasyonel ve adli gereksi...",https://www.sans.org/information-security-poli...
1,Log Management Policy,logging,Log Review and Accountability,Organizations must define processes for regula...,"Kuruluşlar, düzenli log incelemesi için soruml...",https://www.sans.org/information-security-poli...
2,Log Management Policy,logging,Centralized Logging and Missing Logs,Organizations must maintain systems that aggre...,"Kuruluşlar, tüm bilgi sistemlerinden logları t...",https://www.sans.org/information-security-poli...
3,Log Management Policy,logging,Log Integrity and Protection,Log management practices aim to maintain the i...,"Log yönetimi uygulamaları, kayıtların bütünlüğ...",https://www.sans.org/information-security-poli...
4,Log Management Policy,logging,Purpose of Log Management,Log management establishes a structured approa...,"Log yönetimi; sistemler, uygulamalar ve ağ alt...",https://www.sans.org/information-security-poli...


In [ ]:
import pandas as pd

MASTER_URL = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL.csv"
EXTRA_PATH = "/content/extra_chunks_clean.csv"
OUT_PATH = "/content/bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL_plus_extra.csv"

base = pd.read_csv(MASTER_URL)
base.columns = [c.strip() for c in base.columns]

extra = pd.read_csv(EXTRA_PATH)
extra.columns = [c.strip() for c in extra.columns]

# master şemasına map et
mapped = pd.DataFrame({
    "filename": extra["doc_title"].astype(str).str.strip(),
    "source_url": extra["source_url"].astype(str).str.strip(),
    "section_label": extra["section_title"].astype(str).str.strip(),
    "section_index": 0,  # SANS için sabit
    "chunk_index": 0,    # aşağıda üretilecek
    "chunk_text": extra["chunk_text_en"].astype(str),
    "chunk_text_tr": extra["chunk_text_tr"].astype(str),
})

mapped["chunk_index"] = mapped.groupby("filename").cumcount()

merged = pd.concat([base, mapped], ignore_index=True)

# duplicate temizle
merged = merged.drop_duplicates(
    subset=["filename", "section_label", "chunk_text_tr"],
    keep="first"
).reset_index(drop=True)

merged.to_csv(OUT_PATH, index=False, encoding="utf-8")

print("✅ Birleştirildi:", OUT_PATH)
print("Toplam satır:", len(merged))


✅ Birleştirildi: /content/bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL_plus_extra.csv
Toplam satır: 260
